In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ["LLAMA_INDEX_CACHE_DIR"] = '/mnt/nfs/renjiyuan/HF_cache'
os.environ['HF_HOME'] = '/mnt/nfs/renjiyuan/HF_cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = '/mnt/nfs/renjiyuan/HF_cache'

In [2]:
# Setup AzureOpenAI Agent
from llama_index.llms.azure_openai import AzureOpenAI

llm = AzureOpenAI(
    engine="gpt-4",
    model="gpt-4",
    temperature=0.0
)

/home/renjiyuan/miniconda3/envs/rag_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# test llm api
completion_response = llm.complete("To infinity, and")
print('Test api:\n', completion_response)

Test api:
 beyond! 

This phrase is the famous catchphrase of Buzz Lightyear, a character from the Pixar animated film series "Toy Story." It encapsulates the character's adventurous spirit and his belief that there are no limits to what one can achieve.


In [4]:
from llama_index.core.tools.download import download_tool
from llama_index.core.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec
# download google search tool
GoogleSearchToolSpec = download_tool('GoogleSearchToolSpec')

# setup GoogleSearchToolSpec
# key and engine could be found in https://developers.google.com/custom-search/v1/introduction?hl=zh-cn
engine = ''
key = ''
google_spec = GoogleSearchToolSpec(key=key, engine=engine)

# prepare tools
tools = LoadAndSearchToolSpec.from_defaults(
    google_spec.to_tool_list()[0],
).to_tool_list()

/tmp/ipykernel_156541/3935470633.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) download_tool. (`download_tool()` is deprecated. Please install tool using pip install directly instead.)
  GoogleSearchToolSpec = download_tool('GoogleSearchToolSpec')


In [5]:
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core import Settings

# setting the embed model, for default embed model is openai.
# If you have the key of openai, you can set it in the env and skip this step.
Settings.embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")
Settings.llm = llm

In [6]:
from llama_index.core.agent import AgentRunner
# this method will automatically pick the best agent depending on the LLM
agent = AgentRunner.from_llm(tools, llm=llm, verbose=True)

# or
# from llama_index.core.agent import ReActAgent
# agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

In [7]:
# test 
agent.chat("What is DI-engine?")

Added user message to memory: What is DI-engine?
=== Calling Function ===
Calling function: google_search with args: {"query":"DI-engine"}
Got output: Content loaded! You can now search the information using read_google_search

=== Calling Function ===
Calling function: read_google_search with args: {"query":"What is DI-engine?"}
Got output: DI-engine is a generalized decision intelligence engine for PyTorch and JAX, designed to be Python-first. It is an open-source project hosted on GitHub, where developers can contribute to its development. The engine is part of the OpenDILab Decision AI Engine initiative.



AgentChatResponse(response='DI-engine is a generalized decision intelligence engine that supports PyTorch and JAX, designed with a Python-first approach. It is an open-source project available on GitHub, and it is part of the OpenDILab Decision AI Engine initiative, which encourages contributions from developers to its development.', sources=[ToolOutput(content='Content loaded! You can now search the information using read_google_search', tool_name='google_search', raw_input={'args': (), 'kwargs': {'query': 'DI-engine'}}, raw_output='Content loaded! You can now search the information using read_google_search', is_error=False), ToolOutput(content='DI-engine is a generalized decision intelligence engine for PyTorch and JAX, designed to be Python-first. It is an open-source project hosted on GitHub, where developers can contribute to its development. The engine is part of the OpenDILab Decision AI Engine initiative.', tool_name='read_google_search', raw_input={'args': (), 'kwargs': {'quer